# 1. 라이브러리 불러오기

In [23]:
import requests
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import os


from openai import OpenAI
from IPython.display import Image
import urllib
import base64
import io
from PIL import Image
import random

In [8]:
# dotenv로 stability_ai_api_key 가져오기
load_dotenv()
stability_key = os.getenv('STABILITYAI_API_KEY')
# 환경 변수에서 API Key 읽기
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI API Key 설정
OpenAI.api_key = api_key

# print(stability_key)

# 2. 생성

## 2.0. 전처리

In [12]:
# 이미지 결합 함수

def image_concater(first_image, second_image, file_nm, hv):
    # 이미지 열기
    image1 = Image.open(first_image)
    image2 = Image.open(second_image)
    
    # 가로 버젼
    if hv == 'h':
        # 이미지 합치기 (가로)
        new_image_width = image1.width + image2.width
        new_image_height = max(image1.height, image2.height)
        combined_image = Image.new('RGB', (new_image_width, new_image_height))

        # 이미지 붙이기
        combined_image.paste(image1, (0, 0))
        combined_image.paste(image2, (image1.width, 0))

    # 세로 버젼
    elif hv == 'v':
        # 이미지 합치기 (세로)
        new_image_height = image1.height + image2.height
        new_image_width = max(image1.width, image2.width)
        combined_image = Image.new('RGB', (new_image_width, new_image_height))

        # 이미지 붙이기
        combined_image.paste(image1, (0, 0))
        combined_image.paste(image2, (0, image1.height))

    combined_image.show()
    # 합친 이미지를 파일로 저장
    # 폴더 생성하기
    os.makedirs(f"88_output/{file_nm}", exist_ok=True)
    combined_image.save(f"88_output/{file_nm}/{file_nm}.png")
    print("Combined image saved")

In [14]:
# 이미지 파일 경로
first_image_path = "00_ref/03_materials/111630_67539_4056.jpg"
second_image_path = "00_ref/03_materials/image2.webp"

image_concater(first_image_path, second_image_path, "materials", 'h')

Combined image saved


## 2.1. Image

In [3]:
response = requests.post(
    f"https://api.stability.ai/v2beta/stable-image/generate/ultra",
    headers={
        "authorization": stability_key,
        "accept": "image/*"
    },
    files={"none": ''},
    data={
        "prompt": "Lighthouse on a cliff overlooking the ocean",
        "output_format": "png",
    },
)

if response.status_code == 200:
    with open("./lighthouse.webp", 'wb') as file:
        file.write(response.content)
else:
    raise Exception(str(response.json()))

## 2.2. Style

In [33]:
# 카운터
counter = 0

In [38]:
# SET
import datetime

def set_image(image_nm):
    # 현재 시간을 기반으로 파일 이름 생성
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S%f")
    output_path = f"88_output/{image_nm}/{image_nm}_{timestamp}.png"

    prompt = ''' 
    You are a professional furniture designer. You are tasked with creating a collection that blends two different materials shown in the reference images. 
    Create a new furniture piece that combines the two materials in a harmonious way. Chair, Table, or Lamp etc you could design any furniture piece.
    The design should feature a flat top and incorporate free-flowing curves.
    Produce a single image of the newly designed furniture piece, set against a white background, styled as a thumbnail suitable for furniture sales.
    '''

    image_path = f"88_output/{image_nm}/{image_nm}.png"
    response = requests.post(
        "https://api.stability.ai/v2beta/stable-image/control/style",
        headers={
            "authorization": stability_key,
            "accept": "image/*"
        },
        files={
            "image": open(image_path, "rb"),
        },
        data={
            "prompt": prompt,
            "output_format": "png"
        },
    )

    if response.status_code == 200:
        with open(output_path, 'wb') as file:
            file.write(response.content)
        print(f"Image saved successfully: {output_path}")
    else:
        raise Exception(str(response.json()))

In [66]:
# STOOL
prompt = ''' 
You are a professional furniture designer. In this image there are two different furnitures.
Use the reference images and Mix two different materials in reference images to design a stool with a flat top and lots of free-curve(0.8).
Create just ONE(0.9) furniture image of the newly designed furniture with a white(0.8) background that looks like a thumbnail for selling furniture.
'''

image_nm = "ruggys"
response = requests.post(
    f"https://api.stability.ai/v2beta/stable-image/control/style",
    headers={
        "authorization": stability_key,
        "accept": "image/*"
    },
    files={
        "image": open(f"88_output/{image_nm}.png", "rb"),
    },
    data={
        "prompt": prompt,
        "output_format": "png"
    },
)

if response.status_code == 200:
    counter += 1
    with open(f"88_output/{image_nm}_{counter}.png", 'wb') as file:
        file.write(response.content)
else:
    raise Exception(str(response.json()))

In [67]:
# COFFEE TABLE

prompt = ''' 
You are a professional furniture designer. In this image there are two different furnitures.
Use the reference images to design a Coffee Table with a flat top and free curve.
Create a single furniture image of the newly designed furniture with a white(0.8) background that looks like a thumbnail for selling furniture.
'''

image_nm = "ruggys"
response = requests.post(
    f"https://api.stability.ai/v2beta/stable-image/control/style",
    headers={
        "authorization": stability_key,
        "accept": "image/*"
    },
    files={
        "image": open(f"88_output/{image_nm}.png", "rb"),
    },
    data={
        "prompt": prompt,
        "output_format": "png"
    },
)

if response.status_code == 200:
    counter += 1
    with open(f"88_output/{image_nm}_{counter}.png", 'wb') as file:
        file.write(response.content)
else:
    raise Exception(str(response.json()))

- 각 모델별로 스타일을 부여해보면 어떨까? 오히려 모더니즘 적인 사진을 넣어주는 게 좋을 수 있다. 
- 사진 별로 나뉘게 되는 것일까? 사진 풀에서 랜덤으로 고르거나, 프롬프트 엔지니어링 하면 어떨까? 좋은 엔지니어링 사례들을 보면 좋을 것

In [35]:
# 이미지 세트를 기반으로 이미지 생성
# 1. 이미지 폴더 불러오고 거기서 무작위로 이미지 두 개를 선택
# 2. 두 이미지를 결합
# 3. 결합된 이미지를 Stability AI에 전송
# 4. 결과 이미지를 저장

def random_pair(image_set):
    # 이미지 폴더 불러오기
    image_folder = f"00_ref/{image_set}"
    image_files = os.listdir(image_folder)

    # 이미지 두 개를 무작위로 선택
    first_image = random.choice(image_files)
    image_files.remove(first_image)
    second_image = random.choice(image_files)

    # 이미지 결합
    try:
        image_concater(f"{image_folder}/{first_image}", f"{image_folder}/{second_image}", image_set, 'h')
    except:
        image_concater(f"{image_folder}/{first_image}", f"{image_folder}/{second_image}", image_set, 'v')

# 이미지 생성
random_pair("modernism")

Combined image saved


In [39]:
# 가구 세트 생성
set_image("modernism")

Image saved successfully: 88_output/modernism/modernism_20240913_021139370672.png


## 2.2.1. Adding Background

In [22]:
furniture_nm = 'e520c98d560883193df2d112940cb6a1_1'


response = requests.post(
    f"https://api.stability.ai/v2beta/stable-image/control/style",
    headers={
        "authorization": stability_key,
        "accept": "image/*"
    },
    files={
        # "image": open(f"00_ref/{furniture_nm}.png", "rb"),
        "image": open(f"88_output/{furniture_nm}.png", "rb")
    },
    data={
        "prompt": "Retain the furniture piece, stool(1) in its current form and position. Change only the background(1) to resemble a clean, bright yellow(1) studio setting with even lighting. Ensure the background change does not alter the appearance, color, or details of the furniture. The focus should remain on the furniture, with the background subtly enhancing its visibility. Ensure that shadows, reflections, and lighting are consistent with a white studio environment.",
        "output_format": "png",
        "negative_prompt": "DO NOT ADD any other furnitures. Do not alter, distort, or change the furniture itself in any way. Do not add any elements to the furniture or modify its appearance."
    },
)

if response.status_code == 200:
    counter += 1
    with open(f"88_output/{furniture_nm}_{counter}.png", 'wb') as file:
        file.write(response.content)
else:
    raise Exception(str(response.json()))

### 2.2.1 Animal inspired furniture

In [17]:
output_nm = 0

In [24]:

animal_prompt = '''
You are a professional furniture designer, you are designing a modern stool with a design from the image.
The image is animal, and you are going to make a stool inspired by the animal.
But Don't use the animal image directly into the furniture design, Just use the animal image as an inspiration.
Only create the furniture image with the white background which looks like a selling furniture thumbnail.
'''

response = requests.post(
    f"https://api.stability.ai/v2beta/stable-image/control/style",
    headers={
        "authorization": stability_key,
        "accept": "image/*"
    },
    files={
        "image": open("00_ref/01_animal/tican_1.jpg", "rb"),
    },
    data={
        "prompt": animal_prompt,
        "negative_prompt" : 'Never use the animal image directly into the furniture design.',
        "output_format": "png"
    },
)

if response.status_code == 200:
    counter += 1
    with open(f"88_output/set_{counter}.png", 'wb') as file:
        file.write(response.content)
else:
    raise Exception(str(response.json()))

## 2.3. Stable Fast 3D

In [12]:
response = requests.post(
    f"https://api.stability.ai/v2beta/3d/stable-fast-3d",
    headers={
        "authorization": stability_key,
    },
    files={
        "image": open("00_ref/8b3886b2a901acf7e88e96786d4e0448.png", "rb")
    },
    data={},
)

if response.status_code == 200:
    with open("88_output/3D/3d-furniture-test.glb", 'wb') as file:
        file.write(response.content)
else:
    raise Exception(str(response.json()))

## 2.4. 배경 지우기

In [36]:
img_nm = 'stone&silver_12'
response = requests.post(
    f"https://api.stability.ai/v2beta/stable-image/edit/remove-background",
    headers={
        "authorization": stability_key,
        "accept": "image/*"
    },
    files={
        "image": open("88_output/stone&silver_12.png", "rb")
    },
    data={
        "output_format": "png"
    },
)

if response.status_code == 200:
    with open(f"88_output/{img_nm}_back_removed.png", 'wb') as file:
        file.write(response.content)
else:
    raise Exception(str(response.json()))

In [12]:

response = requests.post(
    f"https://api.stability.ai/v2beta/stable-image/edit/outpaint",
    headers={
        "authorization": stability_key,
        "accept": "image/*"
    },
    files={
        "image": open("88_output/back_removed.png", "rb")
    },
    data={
        "left": 200,
        "down": 200,
        "output_format": "png"
    },
)

if response.status_code == 200:
    with open("88_output/back_removed_and_added.png", 'wb') as file:
        file.write(response.content)
else:
    raise Exception(str(response.json()))

In [14]:
# Inpainiting

response = requests.post(
    f"https://api.stability.ai/v2beta/stable-image/edit/inpaint",
    headers={
        "authorization": stability_key,
        "accept": "image/*"
    },
    files={
        "image": open("88_output/ignant-design-kodai-iwamoto-plastic-blowing-01-1440x1700_3.png", "rb"), # 배경
        "mask": open("88_output/back_removed.png", "rb"), # 마스크
    },
    data={
        "prompt": "coffee table",
        "output_format": "png",
    },
)

if response.status_code == 200:
    with open("88_output/filled_with_mask.png", 'wb') as file:
        file.write(response.content)
else:
    raise Exception(str(response.json()))

# 3. 도면화

In [6]:
# 이미지 → 도면 설명글로 전환 
# 상판
def describe_furn(url):
  client = OpenAI()

  prompt = '''
  Please provide a detailed description of the furniture in the image, including materials, colors, sizes, and thicknesses for each part of the furniture. All measurements should be in millimeters (mm). Prioritize the top surface of the furniture for initial drafting. Describe the shape of the top surface as accurately as possible, including all dimensions and details.

  Focus on each part of the furniture, providing specific information on:

  1. Top Surface: Material, color, shape, dimensions, and thickness.
  2. Legs: Material, color, height, width, and thickness.
  3. Frame: Material, color, dimensions, and thickness.
  4. Additional Elements: Any other parts, like shelves or drawers, with their materials, colors, dimensions, and thicknesses.

  Make sure the description is precise, detailed, and comprehensive to ensure the drawing captures all the essential characteristics of the furniture.
  '''

  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {"role": "system", "content": "You are a furniture designer."},
      {
        "role": "user",
        "content": [
          {"type": "text", "text": prompt},
          {
            "type": "image_url",
            "image_url": {
              "url":url,
            },
          },
        ],
      }
    ],
    max_tokens=500,
  )

  return response.choices[0].message.content


In [ ]:
# 출력
description = response.choices[0].message.content
print(description)

In [7]:
def create_furniture_top_view(description):
    drawing_prompt = '''
    With the description provided,I want to create a top-view drawing of the furniture in the image using shapely.
    And the visualizing with matplotlib. In a code format.
    The drawing should be based on the detailed description of each part of the furniture, including the materials, colors, sizes, and thicknesses in millimeters (mm).
    1. Draw subplots for each part of the furniture.
    2. Draw the top surface, legs, frame, and additional elements.
    3. Add dimensions and labels to each part of the furniture.
    4. Add the description at the side about colors and line styles to distinguish each component.
    '''

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a professional furniture designer."},
            {
                "role": "user",
                "content": description + drawing_prompt
            }
        ],
        stream=True
    )

    for chunk in completion:
        print(
            chunk.choices[0].delta.content,
            end=""
            )


In [8]:
# 두번째 이미지
url = 'https://i.postimg.cc/FKCw2d5W/bbca96cf1dde0388e65df2867a54e9c1.jpg'
describe = describe_furn(url)
describe

"Certainly! Here's a detailed description of the coffee table in the image:\n\n### 1. Top Surface:\n- **Material:** Marble or a similar stone material with a natural, polished finish.\n- **Color:** Light beige with subtle marble veining patterns.\n- **Shape:** Rectangular.\n- **Dimensions:**\n  - **Length:** Approximately 1200 mm\n  - **Width:** Approximately 800 mm\n  - **Thickness:** Approximately 40 mm\n- **Details:** The top surface has a smooth, polished finish with natural stone patterning. The corners are sharp, and there are no visible rounded edges.\n\n### 2. Legs:\n- **Material:** Metal, likely steel or iron with a matte finish.\n- **Color:** Black.\n- **Shape and Dimensions:**\n  - **Height:** Approximately 400 mm\n  - **Width (each leg):** 100 mm\n  - **Thickness:** 40 mm\n  - **Design:** The legs have a modern, geometric U-shape, with part of the leg structure extending over the top surface for an integrated look. This creates an L-shaped profile when viewed from the side.

In [ ]:
create_furniture_top_view(describe)

In [ ]:
# Plot 2번째 이미지
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Function to add a rectangle to a subplot
def add_rectangle(ax, xy, width, height, **kwargs):
    rect = patches.Rectangle(xy, width, height, **kwargs)
    ax.add_patch(rect)


# Set up the drawing space
fig, ax = plt.subplots()

# Define the dimensions
top_surface_length = 1200
top_surface_width = 800
top_surface_thickness = 40
leg_height = 300
leg_width = 100
leg_thickness = 20
leg_horizontal_extension = 100

# Define colors and materials
color_top_surface = 'beige'
color_vein = 'brown'
color_legs = 'black'

# Define the top surface
top_surface = Polygon([
    (0, 0),
    (top_surface_length, 0),
    (top_surface_length, top_surface_width),
    (0, top_surface_width)
])

# Define legs positions
legs = [
    (0, 0),
    (top_surface_length - leg_width, 0),
    (top_surface_length - leg_width, top_surface_width - leg_width),
    (0, top_surface_width - leg_width)
]

# Plot the top surface
add_rectangle(ax, (0, 0), top_surface_length, top_surface_width, edgecolor='brown', facecolor='beige', linewidth=2, linestyle='-')

# Plot L-shaped legs
for leg in legs:
    add_rectangle(ax, leg, leg_horizontal_extension, leg_thickness, edgecolor='black', facecolor='black')
    add_rectangle(ax, leg, leg_thickness, leg_height, edgecolor='black', facecolor='black')

# Add labels and annotations
ax.text(top_surface_length / 2, -50, f"{top_surface_length} mm", ha='center', va='center')
ax.text(top_surface_length + 20, top_surface_width / 2, f"{top_surface_width} mm", ha='center', va='center', rotation=90)

for leg in legs:
    ax.text(leg[0] + leg_horizontal_extension / 2, leg[1] + leg_thickness + 10, f"Leg Extension: {leg_horizontal_extension} mm", ha='center')
    ax.text(leg[0] + leg_thickness + 10, leg[1] + leg_height / 2, f"Leg Height: {leg_height} mm", ha='center', rotation=90)

# Configure plot limits and aspect ratio
ax.set_xlim(-100, top_surface_length + 100)
ax.set_ylim(-100, top_surface_width + 100)
ax.set_aspect('equal', 'box')
ax.axis('off')

# Add description of colors and elements
ax.legend([
    patches.Patch(color='beige', label='Top Surface: Faux Marble'),
    patches.Patch(color='black', label='Legs: Matte Black Metal')
], loc='upper right')

# Show plot
plt.show()
